# Sieci neuronowe rekurencyjne

W poprzednim module korzystaliśmy z bogatych reprezentacji semantycznych tekstu oraz prostego klasyfikatora liniowego na bazie osadzeń. Taka architektura pozwala uchwycić zagregowane znaczenie słów w zdaniu, ale nie uwzględnia **kolejności** słów, ponieważ operacja agregacji na osadzeniach usuwa tę informację z oryginalnego tekstu. Ponieważ te modele nie potrafią modelować kolejności słów, nie są w stanie rozwiązywać bardziej złożonych lub niejednoznacznych zadań, takich jak generowanie tekstu czy odpowiadanie na pytania.

Aby uchwycić znaczenie sekwencji tekstu, musimy użyć innej architektury sieci neuronowej, zwanej **siecią neuronową rekurencyjną** (RNN). W RNN przekazujemy nasze zdanie przez sieć, symbol po symbolu, a sieć generuje pewien **stan**, który następnie przekazujemy z kolejnym symbolem z powrotem do sieci.

Given the input sequence of tokens $X_0,\dots,X_n$, RNN creates a sequence of neural network blocks, and trains this sequence end-to-end using back propagation. Each network block takes a pair $(X_i,S_i)$ as an input, and produces $S_{i+1}$ as a result. Final state $S_n$ or output $X_n$ goes into a linear classifier to produce the result. All network blocks share the same weights, and are trained end-to-end using one back propagation pass.

Ponieważ wektory stanów $S_0,\dots,S_n$ są przekazywane przez sieć, jest ona w stanie nauczyć się zależności sekwencyjnych między słowami. Na przykład, gdy słowo *not* pojawia się gdzieś w sekwencji, sieć może nauczyć się negować pewne elementy wektora stanu, co prowadzi do negacji.

> Ponieważ wagi wszystkich bloków RNN na obrazku są współdzielone, ten sam obrazek można przedstawić jako jeden blok (po prawej) z rekurencyjną pętlą sprzężenia zwrotnego, która przekazuje wyjściowy stan sieci z powrotem na wejście.

Zobaczmy, jak sieci neuronowe rekurencyjne mogą pomóc nam w klasyfikacji naszego zbioru danych z wiadomościami.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Prosty klasyfikator RNN

W przypadku prostego RNN każda jednostka rekurencyjna jest prostą siecią liniową, która przyjmuje połączony wektor wejściowy i wektor stanu, a następnie generuje nowy wektor stanu. PyTorch reprezentuje tę jednostkę za pomocą klasy `RNNCell`, a sieci takich komórek - jako warstwę `RNN`.

Aby zdefiniować klasyfikator RNN, najpierw zastosujemy warstwę osadzania (embedding layer), aby zmniejszyć wymiarowość słownika wejściowego, a następnie umieścimy na niej warstwę RNN:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Note:** Używamy tutaj nieprzeszkolonej warstwy osadzania dla uproszczenia, ale dla jeszcze lepszych wyników możemy użyć wstępnie przeszkolonej warstwy osadzania z osadzeniami Word2Vec lub GloVe, jak opisano w poprzedniej jednostce. Aby lepiej zrozumieć, możesz dostosować ten kod do pracy z wstępnie przeszkolonymi osadzeniami.

W naszym przypadku użyjemy załadowanego danych z wypełnieniem, więc każda paczka będzie zawierać kilka wypełnionych sekwencji o tej samej długości. Warstwa RNN przyjmie sekwencję tensorów osadzeń i wygeneruje dwa wyjścia:
* $x$ to sekwencja wyników komórek RNN na każdym kroku
* $h$ to końcowy stan ukryty dla ostatniego elementu sekwencji

Następnie stosujemy w pełni połączony klasyfikator liniowy, aby uzyskać liczbę klas.

> **Note:** Trenowanie RNN jest dość trudne, ponieważ po rozwinięciu komórek RNN wzdłuż długości sekwencji liczba warstw zaangażowanych w propagację wsteczną jest dość duża. Dlatego musimy wybrać małą szybkość uczenia się i trenować sieć na większym zbiorze danych, aby uzyskać dobre wyniki. Może to zająć dużo czasu, więc preferowane jest użycie GPU.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Długoterminowa Pamięć Krótkoterminowa (LSTM)

Jednym z głównych problemów klasycznych RNN jest tzw. problem **zanikających gradientów**. Ponieważ RNN są trenowane end-to-end w jednym przebiegu wstecznej propagacji, mają trudności z propagowaniem błędu do pierwszych warstw sieci, co uniemożliwia sieci naukę relacji między odległymi tokenami. Jednym ze sposobów na uniknięcie tego problemu jest wprowadzenie **jawnego zarządzania stanem** poprzez zastosowanie tzw. **bramek**. Istnieją dwie najbardziej znane architektury tego typu: **Długoterminowa Pamięć Krótkoterminowa** (LSTM) oraz **Jednostka Bramkowana (GRU)**.

![Obraz przedstawiający przykład komórki LSTM](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Sieć LSTM jest zorganizowana w sposób podobny do RNN, ale przekazywane są dwa stany z warstwy do warstwy: aktualny stan $c$ oraz ukryty wektor $h$. W każdej jednostce ukryty wektor $h_i$ jest konkatenowany z wejściem $x_i$, a następnie kontrolują one, co dzieje się ze stanem $c$ za pomocą **bramek**. Każda bramka to sieć neuronowa z aktywacją sigmoidalną (wyjście w zakresie $[0,1]$), którą można traktować jako maskę bitową, gdy jest mnożona przez wektor stanu. Na powyższym obrazku (od lewej do prawej) znajdują się następujące bramki:
* **bramka zapominania** przyjmuje ukryty wektor i określa, które komponenty wektora $c$ należy zapomnieć, a które przepuścić dalej.
* **bramka wejściowa** pobiera pewne informacje z wejścia i ukrytego wektora, a następnie wprowadza je do stanu.
* **bramka wyjściowa** przekształca stan za pomocą pewnej warstwy liniowej z aktywacją $\tanh$, a następnie wybiera niektóre z jej komponentów, używając ukrytego wektora $h_i$, aby wygenerować nowy stan $c_{i+1}$.

Komponenty stanu $c$ można traktować jako pewne flagi, które można włączać i wyłączać. Na przykład, gdy w sekwencji napotkamy imię *Alice*, możemy założyć, że odnosi się ono do postaci żeńskiej, i podnieść flagę w stanie, że w zdaniu występuje rzeczownik żeński. Gdy później napotkamy frazę *and Tom*, podniesiemy flagę, że mamy rzeczownik w liczbie mnogiej. W ten sposób, manipulując stanem, możemy teoretycznie śledzić właściwości gramatyczne części zdania.

> **Note**: Świetnym źródłem do zrozumienia wewnętrznej struktury LSTM jest ten doskonały artykuł [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) autorstwa Christophera Olaha.

Chociaż wewnętrzna struktura komórki LSTM może wydawać się skomplikowana, PyTorch ukrywa tę implementację w klasie `LSTMCell` i udostępnia obiekt `LSTM` do reprezentowania całej warstwy LSTM. W związku z tym implementacja klasyfikatora LSTM będzie bardzo podobna do prostego RNN, który widzieliśmy powyżej:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Sekwencje spakowane

W naszym przykładzie musieliśmy uzupełnić wszystkie sekwencje w minibatchu zerowymi wektorami. Choć prowadzi to do pewnego marnotrawstwa pamięci, w przypadku RNN-ów bardziej istotne jest to, że dodatkowe komórki RNN są tworzone dla uzupełnionych elementów wejściowych, które biorą udział w treningu, ale nie niosą żadnych istotnych informacji wejściowych. Znacznie lepiej byłoby trenować RNN tylko do rzeczywistej długości sekwencji.

Aby to osiągnąć, w PyTorch wprowadzono specjalny format przechowywania uzupełnionych sekwencji. Załóżmy, że mamy uzupełniony minibatch wejściowy, który wygląda tak:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Tutaj 0 reprezentuje wartości uzupełniające, a rzeczywisty wektor długości sekwencji wejściowych to `[5,3,1]`.

Aby efektywnie trenować RNN z uzupełnionymi sekwencjami, chcemy rozpocząć trening pierwszej grupy komórek RNN z dużym minibatchem (`[1,6,9]`), ale następnie zakończyć przetwarzanie trzeciej sekwencji i kontynuować trening z mniejszymi minibatchami (`[2,7]`, `[3,8]`), i tak dalej. W ten sposób spakowana sekwencja jest reprezentowana jako jeden wektor - w naszym przypadku `[1,6,9,2,7,3,8,4,5]`, oraz wektor długości (`[5,3,1]`), z którego można łatwo odtworzyć oryginalny uzupełniony minibatch.

Aby stworzyć spakowaną sekwencję, możemy użyć funkcji `torch.nn.utils.rnn.pack_padded_sequence`. Wszystkie warstwy rekurencyjne, w tym RNN, LSTM i GRU, obsługują spakowane sekwencje jako wejście i produkują spakowane wyjście, które można zdekodować za pomocą `torch.nn.utils.rnn.pad_packed_sequence`.

Aby móc stworzyć spakowaną sekwencję, musimy przekazać wektor długości do sieci, a zatem potrzebujemy innej funkcji do przygotowania minibatchy:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Rzeczywista sieć byłaby bardzo podobna do `LSTMClassifier` powyżej, ale w metodzie `forward` otrzyma zarówno wyściełaną mini-partię, jak i wektor długości sekwencji. Po obliczeniu osadzenia, obliczamy zapakowaną sekwencję, przekazujemy ją do warstwy LSTM, a następnie rozpakowujemy wynik z powrotem.

> **Uwaga**: W rzeczywistości nie używamy rozpakowanego wyniku `x`, ponieważ w dalszych obliczeniach korzystamy z wyjścia z warstw ukrytych. Dlatego możemy całkowicie usunąć rozpakowywanie z tego kodu. Powód, dla którego umieszczamy je tutaj, to umożliwienie łatwej modyfikacji tego kodu, jeśli zajdzie potrzeba użycia wyjścia sieci w dalszych obliczeniach.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Uwaga:** Możesz zauważyć parametr `use_pack_sequence`, który przekazujemy do funkcji treningowej. Obecnie funkcja `pack_padded_sequence` wymaga, aby tensor długości sekwencji znajdował się na urządzeniu CPU, dlatego funkcja treningowa musi unikać przenoszenia danych długości sekwencji na GPU podczas treningu. Możesz zajrzeć do implementacji funkcji `train_emb` w pliku [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## Dwukierunkowe i wielowarstwowe sieci RNN

W naszych przykładach wszystkie sieci rekurencyjne działały w jednym kierunku, od początku sekwencji do jej końca. Wydaje się to naturalne, ponieważ przypomina sposób, w jaki czytamy i słuchamy mowy. Jednak w wielu praktycznych przypadkach mamy losowy dostęp do sekwencji wejściowej, więc może mieć sens przeprowadzenie obliczeń rekurencyjnych w obu kierunkach. Takie sieci nazywane są **dwukierunkowymi** RNN, a można je stworzyć, przekazując parametr `bidirectional=True` do konstruktora RNN/LSTM/GRU.

Pracując z siecią dwukierunkową, potrzebujemy dwóch wektorów stanu ukrytego, po jednym dla każdego kierunku. PyTorch koduje te wektory jako jeden wektor o dwukrotnie większym rozmiarze, co jest bardzo wygodne, ponieważ zazwyczaj przekazujemy wynikowy stan ukryty do w pełni połączonej warstwy liniowej i wystarczy uwzględnić ten wzrost rozmiaru podczas tworzenia warstwy.

Sieć rekurencyjna, jedno- lub dwukierunkowa, wychwytuje określone wzorce w sekwencji i może je przechowywać w wektorze stanu lub przekazywać do wyjścia. Podobnie jak w przypadku sieci konwolucyjnych, możemy zbudować kolejną warstwę rekurencyjną na szczycie pierwszej, aby wychwycić wzorce wyższego poziomu, zbudowane z wzorców niskiego poziomu wyodrębnionych przez pierwszą warstwę. Prowadzi to do pojęcia **wielowarstwowej RNN**, która składa się z dwóch lub więcej sieci rekurencyjnych, gdzie wyjście poprzedniej warstwy jest przekazywane jako wejście do następnej warstwy.

![Obraz przedstawiający wielowarstwową sieć RNN typu LSTM](../../../../../lessons/5-NLP/16-RNN/images/multi-layer-lstm.jpg)

*Obrazek pochodzi z [tego wspaniałego artykułu](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) autorstwa Fernando Lópeza*

PyTorch ułatwia konstruowanie takich sieci, ponieważ wystarczy przekazać parametr `num_layers` do konstruktora RNN/LSTM/GRU, aby automatycznie zbudować kilka warstw rekurencyjnych. Oznacza to również, że rozmiar wektora ukrytego/stanu zwiększy się proporcjonalnie, co należy uwzględnić podczas obsługi wyjścia warstw rekurencyjnych.


## RNN-y do innych zadań

W tej jednostce widzieliśmy, że RNN-y mogą być używane do klasyfikacji sekwencji, ale w rzeczywistości mogą obsługiwać wiele innych zadań, takich jak generowanie tekstu, tłumaczenie maszynowe i inne. Zajmiemy się tymi zadaniami w następnej jednostce.



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
